In [2]:
import tensorflow as tf
import numpy as np
from sklearn import linear_model    

In [3]:
tf.config.run_functions_eagerly(True)

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Import modules
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
import datetime

In [5]:
print(tf.__version__)

2.4.0


## Pre-train

In [6]:
# Lenet
# Reference: https://analyticsindiamag.com/complete-tutorial-on-lenet-5-guide-to-begin-with-cnns/
def build_model(verbose=0):
    train, test = fashion_mnist.load_data()               
    x_train, y_train = train
    x_test, y_test = test 

    # Add a dimension
    x_train = x_train[..., tf.newaxis].astype("float32") #(60000, 28, 28, 1) 
    x_test = x_test[..., tf.newaxis].astype("float32") #(60000,)   

    model = tf.keras.models.Sequential()

    model.add(Conv2D(filters=6, kernel_size=(3, 3), activation='tanh', 
                     input_shape=(28, 28, 1)))
    model.add(AveragePooling2D())
    model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='tanh'))
    model.add(AveragePooling2D())
    model.add(Flatten())
    model.add(Dense(128, activation='tanh'))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(x=x_train, 
              y=y_train,
              batch_size=256,
              epochs=5, 
              validation_data=(x_test, y_test),
              verbose=verbose)
    return model

In [7]:
def get_all_layers():
    for i in model.layers:    
        print(i.name)

In [8]:
def get_weights_bias(model): 
    out = dict()
    out["conv2d"] = model.get_layer('conv2d').get_weights()[0]
    out["bias"] = model.get_layer('conv2d').get_weights()[1]
    out["conv2d_1"] = model.get_layer('conv2d_1').get_weights()[0]
    out["bias_1"] = model.get_layer('conv2d_1').get_weights()[1]
    return out

In [9]:
# build model
model = build_model(verbose=1)

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/5
235/235 [==============================] - 6s 14ms/step - loss: 0.8623 - accuracy: 0.7078 - val_loss: 0.4562 - val_accuracy: 0.8345
Epoch 2/5
235/235 [==============================] - 3s 13ms/step - loss: 0.4123 - accuracy: 0.8503 - val_loss: 0.4110 - val_accuracy: 0.8512
Epoch 3/5
235/235 [==============================] - 3s 13ms/step - loss: 0.3661 - accuracy: 0.8680 - val_loss: 0.3938 - val_accuracy: 0.8506
Epoch 4/5
235/235 [==============================] - 3s 13ms/step - loss: 0.3375 - accuracy: 0.8772 - val_loss: 0.3507 - val_accuracy: 0.8755
Epoch 5/5
235/235 [==============================] - 3s 14ms/step - loss: 0.3193 - accuracy: 0.8842 - val_loss: 0.3635 - val_accuracy: 0.8702


In [10]:
weights = get_weights_bias(model)
print(weights['conv2d'].shape) #10 filters
print(weights['conv2d_1'].shape) #8 filters
print(weights['bias'].shape) #10 biases
print(weights['bias_1'].shape) #8 biases

(3, 3, 1, 6)
(3, 3, 6, 16)
(6,)
(16,)


## Initialize

In [11]:
from layer_func import *

In [12]:
M = 5
B1,_,_ = get_binary_filter(tf.convert_to_tensor(weights['conv2d']),M)
bias1 = tf.convert_to_tensor(weights['bias'])
W1 = tf.convert_to_tensor(weights['conv2d'])

In [13]:
M = 5
B2,_,_ = get_binary_filter(tf.convert_to_tensor(weights['conv2d_1']),M)
bias2 = tf.convert_to_tensor(weights['bias_1'])
W2 = tf.convert_to_tensor(weights['conv2d_1'])

#### Model Setup

In [14]:
def build_model(x_train,y_train,M,N,W1,B1,W2,B2,bias1,bias2,stride=1,verbose=1,lr=0.01):

    model = tf.keras.models.Sequential()

    model.add(ABCLayer(M, N, B1, bias1, W1,stride))
    model.add(AveragePooling2D())
    model.add(ABCLayer(M,N,B2,bias2,W2,stride))
    model.add(AveragePooling2D())
    model.add(Flatten())
    model.add(Dense(128, activation='tanh'))
    model.add(Dense(10, activation='softmax'))
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

#### Train

In [15]:
train, test = fashion_mnist.load_data()                                                    

x_train, y_train = train #(60000, 28, 28,1)
x_test, y_test = test #(10000, 28, 28,1)   

# Add a dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [17]:
M=5
N=3
model_53 = build_model(x_train,y_train,M,N,W1,B1,W2,B2,bias1,bias2,verbose=1,lr=0.001)

In [18]:
model_53.fit(x=x_train, 
          y=y_train,
          batch_size=256,
          epochs=5, 
          validation_data=(x_test, y_test),
          verbose=1)

Epoch 1/5
235/235 [==============================] - ETA: 0s - loss: 1.3937 - accuracy: 0.5472

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


235/235 [==============================] - 22s 84ms/step - loss: 1.3920 - accuracy: 0.5477 - val_loss: 0.7192 - val_accuracy: 0.7410
Epoch 2/5
235/235 [==============================] - 19s 82ms/step - loss: 0.6678 - accuracy: 0.7586 - val_loss: 0.6432 - val_accuracy: 0.7649
Epoch 3/5
235/235 [==============================] - 19s 82ms/step - loss: 0.6119 - accuracy: 0.7757 - val_loss: 0.6201 - val_accuracy: 0.7716
Epoch 4/5
235/235 [==============================] - 20s 83ms/step - loss: 0.5753 - accuracy: 0.7853 - val_loss: 0.5981 - val_accuracy: 0.7744
Epoch 5/5
235/235 [==============================] - 19s 82ms/step - loss: 0.5530 - accuracy: 0.7947 - val_loss: 0.5664 - val_accuracy: 0.7924


In [30]:
model_53.save_weights(filepath="./saved_weights/model_53")

### Experiment with different hyperparameters

M=3, N=3

In [32]:
M = 3
B1,_,_ = get_binary_filter(tf.convert_to_tensor(weights['conv2d']),M)
bias1 = tf.convert_to_tensor(weights['bias'])
W1 = tf.convert_to_tensor(weights['conv2d'])
B2,_,_ = get_binary_filter(tf.convert_to_tensor(weights['conv2d_1']),M)
bias2 = tf.convert_to_tensor(weights['bias_1'])
W2 = tf.convert_to_tensor(weights['conv2d_1'])

In [37]:
N=3
model_33 = build_model(x_train,y_train,M,N,W1,B1,W2,B2,bias1,bias2,verbose=1,lr=0.005)
model_33.fit(x=x_train, y=y_train,batch_size=256,epochs=5, validation_data=(x_test, y_test),verbose=1)

Epoch 1/5
  2/235 [..............................] - ETA: 14s - loss: 2.4471 - accuracy: 0.0693

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


235/235 [==============================] - ETA: 0s - loss: 1.8165 - accuracy: 0.3721

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


235/235 [==============================] - 15s 62ms/step - loss: 1.8143 - accuracy: 0.3728 - val_loss: 0.7931 - val_accuracy: 0.7034
Epoch 2/5
235/235 [==============================] - 15s 63ms/step - loss: 0.7063 - accuracy: 0.7446 - val_loss: 0.6605 - val_accuracy: 0.7601
Epoch 3/5
235/235 [==============================] - 15s 63ms/step - loss: 0.6280 - accuracy: 0.7724 - val_loss: 0.6283 - val_accuracy: 0.7756
Epoch 4/5
235/235 [==============================] - 15s 63ms/step - loss: 0.5855 - accuracy: 0.7872 - val_loss: 0.5844 - val_accuracy: 0.7898
Epoch 5/5
235/235 [==============================] - 15s 63ms/step - loss: 0.5670 - accuracy: 0.7909 - val_loss: 0.5640 - val_accuracy: 0.7968


In [38]:
model_33.save_weights(filepath="./saved_weights/model_33")

M=5, N=5

In [42]:
M = 5
B1,_,_ = get_binary_filter(tf.convert_to_tensor(weights['conv2d']),M)
bias1 = tf.convert_to_tensor(weights['bias'])
W1 = tf.convert_to_tensor(weights['conv2d'])
B2,_,_ = get_binary_filter(tf.convert_to_tensor(weights['conv2d_1']),M)
bias2 = tf.convert_to_tensor(weights['bias_1'])
W2 = tf.convert_to_tensor(weights['conv2d_1'])

In [43]:
N=5
model_55 = build_model(x_train,y_train,M,N,W1,B1,W2,B2,bias1,bias2,verbose=1,lr=0.01)
model_55.fit(x=x_train, y=y_train,batch_size=256,epochs=5, validation_data=(x_test, y_test),verbose=1)

Epoch 1/5
  1/235 [..............................] - ETA: 37s - loss: 2.4248 - accuracy: 0.0898

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


235/235 [==============================] - ETA: 0s - loss: 1.3436 - accuracy: 0.5383

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


235/235 [==============================] - 27s 117ms/step - loss: 1.3417 - accuracy: 0.5388 - val_loss: 0.6366 - val_accuracy: 0.7522
Epoch 2/5
235/235 [==============================] - 27s 115ms/step - loss: 0.5677 - accuracy: 0.7830 - val_loss: 0.5766 - val_accuracy: 0.7690
Epoch 3/5
235/235 [==============================] - 27s 116ms/step - loss: 0.5016 - accuracy: 0.8098 - val_loss: 0.4853 - val_accuracy: 0.8187
Epoch 4/5
235/235 [==============================] - 27s 116ms/step - loss: 0.4738 - accuracy: 0.8226 - val_loss: 0.4859 - val_accuracy: 0.8127
Epoch 5/5
235/235 [==============================] - 27s 115ms/step - loss: 0.4610 - accuracy: 0.8252 - val_loss: 0.5037 - val_accuracy: 0.8153


In [44]:
model_55.save_weights(filepath="./saved_weights/model_55")

### Experiments with different model setups

#### Model with sigmoid binary activation

In [45]:
from layer_func import *

In [46]:
def build_model_sigmoid(x_train,y_train,M,N,W1,B1,W2,B2,bias1,bias2,stride=1,verbose=1,lr=0.01):

    model = tf.keras.models.Sequential()

    model.add(ABCLayer_sigmoid(M, N, B1, bias1, W1,stride))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(ABCLayer_sigmoid(M,N,B2,bias2,W2,stride))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(Flatten())
    model.add(Dense(100, activation='tanh'))
    model.add(Dense(10, activation='softmax'))
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

M=5, N=3

In [47]:
M = 5
B1,_,_ = get_binary_filter(tf.convert_to_tensor(weights['conv2d']),M)
bias1 = tf.convert_to_tensor(weights['bias'])
W1 = tf.convert_to_tensor(weights['conv2d'])
B2,_,_ = get_binary_filter(tf.convert_to_tensor(weights['conv2d_1']),M)
bias2 = tf.convert_to_tensor(weights['bias_1'])
W2 = tf.convert_to_tensor(weights['conv2d_1'])

In [49]:
N=3
model_sigmoid53 = build_model_sigmoid(x_train,y_train,M,N,W1,B1,W2,B2,bias1,bias2,verbose=1,lr=0.001)
model_sigmoid53.fit(x=x_train, y=y_train,batch_size=256,epochs=5, validation_data=(x_test, y_test),verbose=1)

Epoch 1/5
  1/235 [..............................] - ETA: 30s - loss: 2.4627 - accuracy: 0.0742

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


235/235 [==============================] - ETA: 0s - loss: 2.0562 - accuracy: 0.2756

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


235/235 [==============================] - 19s 81ms/step - loss: 2.0546 - accuracy: 0.2763 - val_loss: 1.0554 - val_accuracy: 0.6639
Epoch 2/5
235/235 [==============================] - 19s 81ms/step - loss: 0.9276 - accuracy: 0.6919 - val_loss: 0.7480 - val_accuracy: 0.7241
Epoch 3/5
235/235 [==============================] - 19s 81ms/step - loss: 0.7076 - accuracy: 0.7398 - val_loss: 0.6839 - val_accuracy: 0.7355
Epoch 4/5
235/235 [==============================] - 19s 81ms/step - loss: 0.6492 - accuracy: 0.7564 - val_loss: 0.6579 - val_accuracy: 0.7327
Epoch 5/5
235/235 [==============================] - 19s 80ms/step - loss: 0.6228 - accuracy: 0.7632 - val_loss: 0.6292 - val_accuracy: 0.7639


In [51]:
model_sigmoid53.save_weights(filepath="./saved_weights/model_sigmoid53")

#### Model with different initializations

M=5, N=3

In [52]:
M = 5
B1,_,_ = get_binary_filter(tf.random.normal(shape=weights['conv2d'].shape),M)
bias1 = tf.random.normal(shape=(10,1))
W1 = tf.random.normal(shape=weights['conv2d'].shape)
B2,_,_ = get_binary_filter(tf.random.normal(shape=weights['conv2d_1'].shape),M)
bias2 = tf.random.normal(shape=(8,1))
W2 = tf.random.normal(shape=weights['conv2d_1'].shape)

In [54]:
N=3
model_53_ini = build_model(x_train,y_train,M,N,W1,B1,W2,B2,bias1,bias2,verbose=1,lr=0.01)
model_53_ini.fit(x=x_train, y=y_train,batch_size=256,epochs=5, validation_data=(x_test, y_test),verbose=1)

Epoch 1/5
  1/235 [..............................] - ETA: 30s - loss: 2.2105 - accuracy: 0.1875

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


235/235 [==============================] - ETA: 0s - loss: 0.7767 - accuracy: 0.7223

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


235/235 [==============================] - 19s 81ms/step - loss: 0.7759 - accuracy: 0.7226 - val_loss: 0.4905 - val_accuracy: 0.8150
Epoch 2/5
235/235 [==============================] - 19s 81ms/step - loss: 0.4501 - accuracy: 0.8321 - val_loss: 0.4552 - val_accuracy: 0.8301
Epoch 3/5
235/235 [==============================] - 19s 83ms/step - loss: 0.4070 - accuracy: 0.8491 - val_loss: 0.4724 - val_accuracy: 0.8217
Epoch 4/5
235/235 [==============================] - 19s 83ms/step - loss: 0.3819 - accuracy: 0.8562 - val_loss: 0.4228 - val_accuracy: 0.8465
Epoch 5/5
235/235 [==============================] - 19s 83ms/step - loss: 0.3683 - accuracy: 0.8614 - val_loss: 0.4127 - val_accuracy: 0.8504


In [56]:
model_53_ini.save_weights("./saved_weights/model_53_ini")

In [57]:
!tree ./ >> README.md